In [11]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
dictA=np.load('Pre_processed_Data/mapping_dict.npy', allow_pickle=True).item()
df = pd.DataFrame(list(dictA.items()), columns=["node", "label"])
df['scale0']=df['node']
for j in range(1,4):
    df[f'scale{j}']=df['node'].map({i: f"C{j}_{np.load('Results/robust_partitions_f.npy')[j-1][i]}" for i in range(238)})

    
scales = ["scale0", "scale1", "scale2", 'scale3']

all_coms = pd.unique(df[scales].values.ravel())
com_to_id = {com: i for i, com in enumerate(all_coms)}

unique_labels = df["label"].unique()

palette = pc.qualitative.Bold   


label_to_color = {
    lab: palette[i % len(palette)] for i, lab in enumerate(unique_labels)
}

# map rgb to rgba with transparency
for key in label_to_color:
    rgb = label_to_color[key]
    label_to_color[key] = rgb.replace("rgb", "rgba").replace(")", ",0.4)")

# change color of teachers to dark grey
label_to_color["Teachers"] = "black"

def transitions_by_class(df, from_col, to_col):
    g = (
        df.groupby(["label", from_col, to_col])
          .size()
          .reset_index(name="count")
    )
    g["source"] = g[from_col].map(com_to_id)
    g["target"] = g[to_col].map(com_to_id)
    g["color"] = g["label"].map(label_to_color)
    return g

t0_1 = transitions_by_class(df, "scale0", "scale1")
t1_2 = transitions_by_class(df, "scale1", "scale2")
t2_3 = transitions_by_class(df, "scale2", "scale3")
links = pd.concat([t0_1, t1_2, t2_3], ignore_index=True)

fig = go.Figure()

# Sankey trace
fig.add_trace(go.Sankey(
    node=dict(
        pad=20,
        thickness=10,
        color="rgba(0,0,0,0.1)"
    ),
    link=dict(
        source=links["source"],
        target=links["target"],
        value=links["count"],
        color=links["color"]
    )
))


for lab in unique_labels:
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode="markers",
        marker=dict(size=12, color=label_to_color[lab]),
        name=lab
    ))

fig.update_layout(
    font_size=12,
    legend_title="Class",
        paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    width=400
)

fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

#fig.write_image(f"Figures/robust_sankey_f_classes.svg")

fig.show()




In [ ]:
df['scale0']=df['node']

,node,label,scale0,scale1,scale2
0,0,1A,C0_0,C1_0,C2_0
1,1,1A,C0_0,C1_0,C2_0
2,2,1A,C0_0,C1_0,C2_0
3,3,1A,C0_0,C1_0,C2_0
4,4,1A,C0_0,C1_0,C2_0
...,...,...,...,...,...
233,233,Teachers,C0_4,C1_5,C2_2
234,234,Teachers,C0_2,C1_1,C2_1
235,235,Teachers,C0_7,C1_3,C2_4
236,236,Teachers,C0_8,C1_4,C2_3


In [22]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
dictA=np.load('Pre_processed_Data/mapping_dict.npy', allow_pickle=True).item()
df = pd.DataFrame(list(dictA.items()), columns=["node", "label"])

df[f'scale0']=df['node'].map({i: f"C{j}_{i}" for i in range(238)})

for j in range(3):
    df[f'scale{j+1}']=df['node'].map({i: f"C{j}_{np.load('Results/robust_partitions_f.npy')[j][i]}" for i in range(238)})

    
scales = ["scale0", "scale1", "scale2", "scale3"]

all_coms = pd.unique(df[scales].values.ravel())
com_to_id = {com: i for i, com in enumerate(all_coms)}

unique_labels = df["label"].unique()

palette = pc.qualitative.Bold   


label_to_color = {
    lab: palette[i % len(palette)] for i, lab in enumerate(unique_labels)
}

# map rgb to rgba with transparency
for key in label_to_color:
    rgb = label_to_color[key]
    label_to_color[key] = rgb.replace("rgb", "rgba").replace(")", ",0.4)")

# change color of teachers to dark grey
label_to_color["Teachers"] = "black"

def transitions_by_class(df, from_col, to_col):
    g = (
        df.groupby(["label", from_col, to_col])
          .size()
          .reset_index(name="count")
    )
    g["source"] = g[from_col].map(com_to_id)
    g["target"] = g[to_col].map(com_to_id)
    g["color"] = g["label"].map(label_to_color)
    return g

t0_1 = transitions_by_class(df, "scale0", "scale1")
t1_2 = transitions_by_class(df, "scale1", "scale2")
t2_3 = transitions_by_class(df, "scale2", "scale3")
links = pd.concat([t0_1, t1_2, t2_3], ignore_index=True)

fig = go.Figure()

# Sankey trace
fig.add_trace(go.Sankey(
    node=dict(
        pad=20,
        thickness=20,
        color="rgba(0,0,0,0.1)"
    ),
    link=dict(
        source=links["source"],
        target=links["target"],
        value=links["count"],
        color=links["color"]
    )
))


for lab in unique_labels:
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode="markers",
        marker=dict(size=12, color=label_to_color[lab]),
        name=lab
    ))

fig.update_layout(
    font_size=12,
    legend_title="Class",
        paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    width=400
)

fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.write_image(f"Figures/robust_sankey_f_classes_singletons.svg")

fig.show()


